In [9]:
import pandas as pd
import numpy as np
from pdb import set_trace
import mysql.connector

In [10]:
#work information
Energy_filepath= 'Energy Indicators.xls'
wb_filepath='world_bank.csv'
sci_filepath ='scimagojr-3.xlsx'

In [11]:
#open datasets
energy = pd.read_excel(Energy_filepath)
GDP = pd.read_csv(wb_filepath,skiprows=4)
ScimEn = pd.read_excel(sci_filepath)

In [12]:
#work with first file - energy
#Renaming the following list of countries to modern names 
#Change ... to NaN
energy = (energy[16:243]
.drop(energy.columns[[0, 1]], axis=1)
.rename(columns={'Environmental Indicators: Energy': 'Country',
                 'Unnamed: 3':'Energy Supply',
                 'Unnamed: 4' : 'Energy Supply per Capita',
                 'Unnamed: 5':'% Renewable'})
          .set_index('Country')
          .replace('...',np.nan)
      )

#change energy measure form Petawatt to gigawatt
energy['Energy Supply']*= 1000000


#change rows
rows_to_Change = {"Republic of Korea": "South Korea",
                  "United States of America": "United States",
                  "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                  "China, Hong Kong Special Administrative Region": "Hong Kong"}
#cleaning the names of rows. delete a spesial symbols.
energy = energy.reset_index() 
energy['Country'] = (energy['Country'].apply(lambda x: x.split('(')[0].rstrip(' ').replace(r"\d+",''))
    .apply(lambda a: "".join([x for x in a if any([x.isalpha(),(x==' ')])]))
                     )
energy.replace({"Country": rows_to_Change},inplace = True)


In [13]:
#work with second file - GDP
#change rows names and clead data.
rows_to_Change = {'Korea, Rep.': "South Korea", 
                  "Iran, Islamic Rep.": "Iran",
                  "Hong Kong SAR, China": "Hong Kong"
                 }
#leave only last 10 yeas 2006 - 2016
GDP = (GDP.replace({'Country Name': rows_to_Change})
       .rename(columns={'Country Name': 'Country'})
       .drop(GDP.columns[1:-10],axis=1))

In [15]:
#joining 3 datasets together
df = ScimEn.merge(energy,on='Country').merge(GDP, on='Country').set_index('Country')
len_of_df = len(energy), len(GDP), len(ScimEn), len(df)
#sorting values
df.sort_values(by="Rank",inplace=True)
#return top15 by Rank
Top15 = df[:15]

In [16]:
energy.to_csv('struct_energy.csv')
GDP.to_csv('GDP.csv')
ScimEn.to_csv('ScimEn.csv')
Top15.to_csv('TOP15_air_pol.csv')